# Forecasting the monthly sales of French champagne

In [2]:
import scipy
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn as skl
import statsmodels as stmd
from math import sqrt

series = pd.read_csv('monthly-champagne-sales.csv', header=0)
dataset, validation = series[0:-12], series[-12:] #test set is the final year
dataset.to_csv('dataset.csv')
validation.to_csv('validation.csv')

# preparing data
X = dataset.values
X = X.astype('float32')
train_size = int(len(X) * 0.5)
train, test = X[0:train_size], X[train_size:]

